In [ ]:
# Librerias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import cm
from matplotlib import colors
pd.options.display.max_columns = None

In [ ]:
# Lectura y limpieza de datos
datos=[]
ataque=pd.read_csv('data/attacking.csv').drop(['serial','match_played'],axis=1)
intentos=pd.read_csv('data/attempts.csv').drop(['serial','match_played'],axis=1)
defenza=pd.read_csv('data/defending.csv').drop(['serial','match_played'],axis=1)
pases=pd.read_csv('data/distributon.csv').drop(['serial','match_played'],axis=1)
porteros=pd.read_csv('data/goalkeeping.csv').drop(['serial','match_played'],axis=1)
goles=pd.read_csv('data/goals.csv').drop(['serial','match_played'],axis=1)
general=pd.read_csv('data/key_stats.csv').drop(['goals','assists'],axis=1)
datos=[intentos,defenza,pases,porteros,goles,general]
df=ataque
for i in datos:
    df=pd.merge(df,i,on=['player_name','club','position'],how='outer')
df.drop(df[df['player_name']=='Reinildo'].index,inplace=True)
df.drop(df[df['player_name']=='Burak Yilmaz'].index,inplace=True)
df.drop(df[df['player_name']=='Luis Díaz'].index,inplace=True)
df.drop(df[df['player_name']=='Brobbey'].index,inplace=True)
df['g_p']=df['goals']/df['match_played'] # Cantidad de goles por partidos jugados
df['g_m']=df['minutes_played']/df['goals'] # Cantidad de minutos para que marque un gol
df['a_p']=df['assists']/df['match_played'] # Cantidad de asistencias por partidos jugados
df['a_m']=df['minutes_played']/df['assists'] # Cantidad de minutos para que de una asistencia
df.insert(36,'no_penalties',df['goals']-df['penalties']) # Cantidad de goles que no fueron de penal
df.insert(9,'tiros_porteria',(100*df['on_target'])/df['total_attempts']) # Cantidad de tiros a porteria
df.insert(11,'tiros_no_porteria',(100*df['off_target'])/df['total_attempts']) # Cantidad de tiros sin direccion de porteria
df.insert(13,'tiros_bloqueados',(100*df['blocked'])/df['total_attempts']) # Cantidad de tiros bloqueados
df.insert(27,'salvadas_p',df['saved']/df['match_played']) # Cantidad de goles salvados por partido
df.insert(29,'goles_p',df['conceded']/df['match_played']) # Cantidad de goles permitidos por partido
df.insert(31,'penales_p',df['saved_penalties']/df['match_played']) # Cantidad de penales salvados por partido
df.insert(33,'cero_goles',(df['cleansheets']*100)/df['match_played']) # Cantidad de porterias en ceros por partido
df.fillna(0,inplace=True) # Relleno los valores NaN con 0
df.loc[df['total_attempts'] == 0.0, 'tiros_porteria'] = np.nan # Eliminar los valores 0 de tiros_porteria para que no se concideren en las graficas
df.loc[df['total_attempts'] == 0.0, 'tiros_no_porteria'] = np.nan # Eliminar los valores 0 de tiros_no_porteria para que no se concideren en las graficas
df.loc[df['total_attempts'] == 0.0, 'tiros_bloqueados'] = np.nan # Eliminar los valores 0 de tiros_bloqueados para que no se concideren en las graficas
df.loc[df['goals'] == 0.0, 'g_m'] = np.nan # Eliminar los valores 0 de g_m para que no se concideren en las graficas
df.loc[df['assists'] == 0.0, 'a_m'] = np.nan # Eliminar los valores 0 de a_m para que no se concideren en las graficas
equipos=pd.read_csv('data/club_matches.csv', encoding = 'cp1252').drop(['Unnamed: 0'],axis=1).set_index('club') # Leer partidos de equipos
equipos=pd.merge(equipos,df.groupby(['club'])['goals'].sum(), on='club') # Agregar goles totales por equipo
equipos['g_m']=equipos['goals']/equipos['teams_matches'] # Cantidad de goles por partido de equipos
df=pd.merge(df,equipos['teams_matches'],on='club') # Agregar cantidad de partidos por equipo a registro de jugadores

In [ ]:
# Ver DataFrames
# df.columns
# df
# df.describe()
# equipos.columns
# equipos
# equipos.describe()

# Datos generales
Se presentan las estadisticas totales de la temporada

In [ ]:
# Obtener datos generales, se necesita para las siguientes celdas
totales=df['goals'].sum()
izquierda=round(df['left_foot'].sum()*100/totales,2)
derecha=round(df['right_foot'].sum()*100/totales,2)
cabeza=round(df['headers'].sum()*100/totales,2)
otros=round(df['others'].sum()*100/totales,2)
dentro_del_area=round(df['inside_area'].sum()*100/totales,2)
fuera_del_area=round(df['outside_areas'].sum()*100/totales,2)

In [ ]:
# Mostrar como texto los datos generales
print('Cantidad de equipos: {}'.format(len(pd.unique(df['club']))))
print('Cantidad de jugadores: {}'.format(len(pd.unique(df['player_name']))))
print("---------------------------------------------------")
print('Asistencias totales: {}'.format(int(df['assists'].sum())))
print('Goles totales: {}'.format(totales))
print("---------------------------------------------------")
print('Goles con la izquierda: {} --> %{}'.format(int(df['left_foot'].sum()),izquierda))
print('Goles con la derecha: {} --> %{}'.format(int(df['right_foot'].sum()),derecha))
print('Goles con la cabeza: {} --> %{}'.format(int(df['headers'].sum()),cabeza))
print('Otros: {} --> %{}'.format(int(df['others'].sum()),otros))
print("---------------------------------------------------")
print('Goles dentro del area: {} --> %{}'.format(int(df['inside_area'].sum()),dentro_del_area))
print('Goles fuera del area: {} --> %{}'.format(int(df['outside_areas'].sum()),fuera_del_area))
print("---------------------------------------------------")
print('Goles de penal: {}'.format(int(df['penalties'].sum())))
print("---------------------------------------------------")

In [ ]:
# Grafica de pastel, Tipos de goles
datos=[derecha,izquierda,cabeza,otros]
normdata=colors.Normalize(min(datos),max(datos))
colormap = cm.get_cmap("jet")
colores =colormap(normdata(datos))
separacion=[0.02,0.02,0.02,0.02]
fig, ax = plt.subplots()
ax.pie(datos,colors=colores, explode=separacion,labels=['Derecha','Izquierda','Cabeza','Otros'], autopct="%0.1f %%",labeldistance=1.1,textprops={'color':"azure"})
ax.axis('equal')
plt.title('Tipos de goles',y=1.05,fontdict={'fontsize':20,'color':'azure', 'fontweight':'bold'})
plt.show()

In [ ]:
# Grafica de pastel, Goles dentro y fuera del area
datos=[dentro_del_area,fuera_del_area]
normdata=colors.Normalize(min(datos),max(datos))
colormap = cm.get_cmap("turbo")
colores =colormap(normdata(datos))
separacion=[0.02,0.02]
fig, ax = plt.subplots()
ax.pie(datos,colors=colores, explode=separacion,labels=['Dentro del area','Fuera del area'], autopct="%0.1f %%",labeldistance=1.1,textprops={'color':"azure"})
ax.axis('equal')
plt.title('Goles dentro y fuera del area',y=1.05,fontdict={'fontsize':20,'color':'azure', 'fontweight':'bold'})
plt.show()

# Goleadores

In [ ]:
# Totales
df[['player_name','club','match_played','goals']].sort_values('goals',ascending=False).reset_index(drop=True).query("goals>0")

In [ ]:
# Por partido
df[['player_name','club','g_p']].sort_values('g_p',ascending=False).reset_index(drop=True).query("g_p>0")

In [ ]:
# Por minutos
df[['player_name','club','g_m']].sort_values('g_m',ascending=False).reset_index(drop=True).query("g_m>0")

In [ ]:
# Minutos por gol
df[['player_name','club','m_g']].sort_values('m_g').reset_index(drop=True).query("m_g>0")

In [ ]:
# Equipo total
df.groupby(['club'])['goals'].sum().reset_index().sort_values('goals',ascending=False).reset_index(drop=True)

In [ ]:
# Posicion
df.groupby(['position'])['goals'].sum().reset_index().sort_values('goals',ascending=False).reset_index(drop=True)

In [ ]:
# Equipo y posicion
df.groupby(['club','position'])['goals'].sum().reset_index().sort_values('goals',ascending=False).reset_index(drop=True)

# Asistencias

In [ ]:
# Totales
df[['player_name','club','match_played','assists']].sort_values('assists',ascending=False).reset_index(drop=True).query("assists>0")

In [ ]:
# Por partido
df[['player_name','club','a_p']].sort_values('a_p',ascending=False).reset_index(drop=True).query("a_p>0")

In [ ]:
# Por minutos
df[['player_name','club','a_m']].sort_values('a_m',ascending=False).reset_index(drop=True).query("a_m>0")

In [ ]:
# Posicion
df.groupby(['position'])['assists'].sum().reset_index().sort_values('assists',ascending=False).reset_index(drop=True).query("assists>0")

In [ ]:
# Equipo y posicion
df.groupby(['club','position'])['assists'].sum().reset_index().sort_values('assists',ascending=False).reset_index(drop=True).query("assists>0")

# Buscador

In [ ]:
# Buscador individual
bandera=True
while bandera or df_j.shape[0]==0:
    bandera=False
    jugador=input('Ingrese el nombre del jugador ')
    df_j=df[df['player_name']==jugador].reset_index(drop=True)
    if df_j.shape[0]>1:
        print(df_j[['player_name','club']])
        indice_jugador=int(input(f'Existen {df_j.shape[0]} coincidencias.\nIngresa el numero del jugadar que quieres seleccionar'))
        df_j=df_j.loc[indice_jugador]
posicion=df_j['position'][0]
club=df_j['club'][0]
df_pos=df[df['position']==posicion]
df_club=df_pos[df_pos['club']==club]
dfs=[df_pos,df_club]
# Graficar

# Grafica Distribucion de goles
if df_j['goals'][0]>0: 
    # Datos a graficar
    colores1=cm.get_cmap('Set1', 4).colors
    colores2=cm.get_cmap('Dark2', 2).colors
    colores3=cm.get_cmap('tab10', 2).colors
    x1=df_j.loc[0,'right_foot':'others'][df_j.loc[0,'right_foot':'others']>0]
    x2=df_j.loc[0,'inside_area':'outside_areas'][df_j.loc[0,'inside_area':'outside_areas']>0]
    x3=df_j.loc[0,'penalties':'no_penalties'][df_j.loc[0,'penalties':'no_penalties']>0]
    plt.figure(figsize=(15,10)) # Crear grafica 1
    plt.subplot(111)
    plt.suptitle(f'Distribucion de goles de {jugador}',fontsize=21,fontweight='bold',color='w') # Titulo
    # Crear graficas
    x1.plot.pie(legend=True,colors=colores1,wedgeprops={'linewidth':1.8,'edgecolor':"slategray"},autopct='%1.2f%%',textprops={'color':'lavender','fontsize':10},pctdistance=0.88)
    x2.plot.pie(legend=True,colors=colores2,radius=0.75,startangle=90,wedgeprops={'linewidth':1.8,'edgecolor':"slategray"},autopct='%1.2f%%',textprops={'color':'lavender','fontsize':10},pctdistance=0.88)
    x3.plot.pie(legend=True,colors=colores3,radius=0.5,startangle=90,wedgeprops={'linewidth':1.8,'edgecolor':"slategray"},autopct='%1.2f%%',textprops={'color':'lavender','fontsize':10},pctdistance=0.88)
    # Circulo central
    centre_circle=plt.Circle((0,0),0.25,fc='lightgray')
    fig=plt.gcf()
    fig.gca().add_artist(centre_circle)
    # Texto del circulo
    plt.text(-0.1,0.01,f"Goles: {int(df_j['goals'][0])}",fontdict={'fontweight':'bold','fontsize':12})
    plt.text(-0.15,-0.06,f"Partidos: {int(df_j['match_played'][0])}",fontdict={'fontweight':'bold','fontsize':12})
    plt.axis('equal') # Para que el circulo no tenga forma de elipse
    plt.legend(loc=7,fontsize=12) # Formato de la leyenda
    plt.ylabel('')
    plt.show() # Mostrar  
# Grafica Comparacion por posicion y Comparacion por posicion y equipo
for n in range(2):
    if posicion!='Goalkeeper':
        fig,ax=plt.subplots(nrows=5, ncols=3, figsize=(25,16)) # Crear figura
        if n==0:
            plt.suptitle(f'Comparacion de {jugador} contra jugadores de su posicion ({posicion})',fontsize=21,fontweight='bold',color='w') # Titulo
        else:
            plt.suptitle(f'Comparacion de {jugador} contra jugadores de su posicion ({posicion}) y equipo ({club})',fontsize=21,fontweight='bold',color='w') # Titulo
        columnas=[['goals','g_p','g_m'],['assists','a_p','a_m'],['tiros_porteria','tiros_no_porteria','tiros_bloqueados'],['pass_accuracy','cross_accuracy','offsides'],['corner_taken','freekicks_taken','dribbles']] # Columnas por fila a graficar
        titulos=[['Goles','Goles por partido','Minutos entre goles'],['Asistencias','Asistencias por partido','Minutos entre asistencias'],['% de tiros con direccion a porteria','% de tiros sin direccion a porteria','% de tiros bloqueados']\
                ,['Presicion de pase','Presicion de cambio de juego','Fuera de juego'],['Tiros de esquina','Tiros libres tirados','Regates']] # Titulos de las columna
        # Graficar
        for i,col in zip(range(5),columnas):
            for j in range(3): #datos    posicion    orientacion dar formato        ancho       colores->      caja                bordes caja                    max/min          ancho                          linea central     ancho                  tamaño circulos                       mediana
                dfs[n].boxplot([col[j]], ax=ax[i][j],vert=False, patch_artist=True, widths=0.25,boxprops=dict(facecolor='#A1C6EA', color='#558B6E'),capprops=dict(color='#558B6E', linewidth=3),whiskerprops=dict(color='#558B6E', linewidth=1.7),flierprops=dict(markersize=7),medianprops=dict(color='red'))
                ax[i][j].plot(df_j[col[j]][0],1, marker="D", color="#F4442E",markersize=10) # Datos jugador seleccionado/ figura diamante, color marcador, tamaño marcador
                ax[i][j].text(df_j[col[j]][0],0.75,f"{round(df_j[col[j]][0],2)}",fontdict={'fontsize':15,'color':'#F4442E','fontweight':'bold'},ha='center') # Texto en negritas 
                ax[i][j].tick_params(axis='x',labelsize=15,labelcolor='w')
                ax[i][j].tick_params(axis='y',labelsize=1)
                ax[i][j].set_facecolor('#E6E8E6') # Color de fondo
                ax[i][j].set_title(titulos[i][j],fontdict={'fontsize':18,'color':'#F18F01','fontweight':'bold'},pad=20) # Titulo de cada grafica
            ax[0][1].text(dfs[n]['g_p'].max(),1.3,f"Partidos jugados: {df_j['match_played'][0]}",fontdict={'fontsize':15,'color':'#02182B','fontweight':'semibold'},bbox={'facecolor':'#A0C1B9','alpha':0.3},ha='right') # Texto partidos jugados
            ax[0][2].text(dfs[n]['g_m'].max(),1.3,f"Minutos jugados: {df_j['minutes_played'][0]}",fontdict={'fontsize':15,'color':'#02182B','fontweight':'semibold'},bbox={'facecolor':'#A0C1B9','alpha':0.3},ha='right') # Texto minutos jugados
            ax[2][0].text(dfs[n]['tiros_porteria'].max(),1.3,f"Tiros totales: {df_j['total_attempts'][0]}",fontdict={'fontsize':15,'color':'#02182B','fontweight':'semibold'},bbox={'facecolor':'#A0C1B9','alpha':0.3},ha='right') # Texto tiros totales
            ax[3][0].text(dfs[n]['pass_accuracy'].max(),1.3,f"Pases totales: {df_j['pass_attempted'][0]}",fontdict={'fontsize':15,'color':'#02182B','fontweight':'semibold'},bbox={'facecolor':'#A0C1B9','alpha':0.3},ha='right') # Texto tiros totales
            ax[3][1].text(dfs[n]['cross_accuracy'].max(),1.3,f"Cambios de juego totales: {df_j['cross_attempted'][0]}",fontdict={'fontsize':15,'color':'#02182B','fontweight':'semibold'},bbox={'facecolor':'#A0C1B9','alpha':0.3},ha='right') # Texto tiros totales
        fig.tight_layout()
        plt.show()
    else:
        fig,ax=plt.subplots(nrows=4, ncols=2, figsize=(25,16)) # Crear figura
        if n==0:
            plt.suptitle(f'Comparacion de {jugador} contra jugadores de su posicion ({posicion})',fontsize=21,fontweight='bold',color='w') # Titulo
        else:
            plt.suptitle(f'Comparacion de {jugador} contra jugadores de su posicion ({posicion}) y equipo ({club})',fontsize=21,fontweight='bold',color='w') # Titulo
        columnas=[['saved','salvadas_p'],['conceded','goles_p'],['saved_penalties','penales_p'],['cleansheets','cero_goles']] # Columnas por fila a graficar
        titulos=[['Tiros parados','Tiros parados por partido'],['Goles permitidos','Goles permitidos por partido'],['Penales salvados','Penales salvados por partido'],['Porterias en cero','Porcentaje de porterias en cero']] # Titulos de las columna
        # Graficar
        for i,col in zip(range(4),columnas):
            for j in range(2): #datos    posicion    orientacion dar formato        ancho       colores->      caja                bordes caja                    max/min          ancho                          linea central     ancho                  tamaño circulos                       mediana
                dfs[n].boxplot([col[j]], ax=ax[i][j],vert=False, patch_artist=True, widths=0.25,boxprops=dict(facecolor='#A1C6EA', color='#558B6E'),capprops=dict(color='#558B6E', linewidth=3),whiskerprops=dict(color='#558B6E', linewidth=1.7),flierprops=dict(markersize=7),medianprops=dict(color='red'))
                ax[i][j].plot(df_j[col[j]][0],1, marker="D", color="#F4442E",markersize=10) # Datos jugador seleccionado/ figura diamante, color marcador, tamaño marcador
                ax[i][j].text(df_j[col[j]][0],0.75,f"{round(df_j[col[j]][0],2)}",fontdict={'fontsize':15,'color':'#F4442E','fontweight':'bold'},ha='center') # Texto en negritas 
                ax[i][j].tick_params(axis='x',labelsize=15,labelcolor='w')
                ax[i][j].tick_params(axis='y',labelsize=1)
                ax[i][j].set_facecolor('#E6E8E6') # Color de fondo
                ax[i][j].set_title(titulos[i][j],fontdict={'fontsize':18,'color':'#F18F01','fontweight':'bold'},pad=20) # Titulo de cada grafica
        ax[0][1].text(dfs[n]['salvadas_p'].max(),1.3,f"Partidos jugados: {df_j['match_played'][0]}",fontdict={'fontsize':15,'color':'#02182B','fontweight':'semibold'},bbox={'facecolor':'#A0C1B9','alpha':0.3},ha='right') # Texto partidos jugados
        fig.tight_layout()
        plt.show()